# Goal-based agents

This agent uses simple coordinate math to formulate its plan. It looks at the goal coordinates, calculates how many steps "Down" and "Right" it needs, creates a full list of actions (the plan), and then executes them.

This creates the simplest distinction between a Reflex Agent and a Goal-Based Agent:

**Reflex Agent**: "I see I'm not at the goal. I'll take one step Right." (Repeat 100 times).

**Goal-Based**: "To get to (4,4), I need 4 Down moves and 4 Right moves. Here is my list. Go."

In [ ]:
import time
import os

class Environment:
    """
    Represents a simple 2D Grid.
    Agent 'A' needs to get to Goal 'G'.
    """
    def __init__(self, size, start, goal):
        self.size = size
        self.agent_pos = list(start) # Mutable list for easier updates
        self.goal_pos = list(goal)
        self.path_taken = []

    def update_agent_pos(self, action):
        """Updates the internal state based on the agent's action."""
        if action == "Down":
            self.agent_pos[0] += 1
        elif action == "Up":
            self.agent_pos[0] -= 1
        elif action == "Right":
            self.agent_pos[1] += 1
        elif action == "Left":
            self.agent_pos[1] -= 1

        # Store as tuple for history
        self.path_taken.append(tuple(self.agent_pos))

    def display(self):
        """Prints the grid to the console."""
        # clear screen command (cls for windows, clear for unix)
        os.system('cls' if os.name == 'nt' else 'clear')

        print(f"--- SIMPLE GOAL AGENT ---")
        print(f"Goal: {self.goal_pos} | Current: {self.agent_pos}")
        print("-" * (self.size * 3 + 1))

        for r in range(self.size):
            row_str = "|"
            for c in range(self.size):
                if r == self.agent_pos[0] and c == self.agent_pos[1]:
                    row_str += " A "
                elif r == self.goal_pos[0] and c == self.goal_pos[1]:
                    row_str += " G "
                elif (r,c) in self.path_taken:
                     row_str += " . "
                else:
                    row_str += "   "
            print(row_str + "|")
        print("-" * (self.size * 3 + 1))
        time.sleep(0.8)

In [ ]:
class GoalBasedAgent:
    def __init__(self, env):
        self.env = env
        self.plan = []

    def perceive(self):
        return self.env.agent_pos

    def formulate_goal(self):
        return self.env.goal_pos

    def formulate_plan(self, start, goal):
        """
        Simple Planner (No BFS/DFS)
        Instead of searching a graph, we just calculate the arithmetic difference
        to generate a plan sequence.
        """
        print("Agent is formulating a plan based on coordinates...")
        time.sleep(1)

        plan = []

        # 1. Calculate vertical distance
        diff_row = goal[0] - start[0]
        if diff_row > 0:
            plan.extend(["Down"] * diff_row)
        elif diff_row < 0:
            plan.extend(["Up"] * abs(diff_row))

        # 2. Calculate horizontal distance
        diff_col = goal[1] - start[1]
        if diff_col > 0:
            plan.extend(["Right"] * diff_col)
        elif diff_col < 0:
            plan.extend(["Left"] * abs(diff_col))

        return plan

    def run(self):
        # 1. PERCEIVE
        current_loc = self.perceive()
        goal_loc = self.formulate_goal()

        # 2. PLAN
        # The key difference: We generate the FULL plan before taking a single step.
        self.plan = self.formulate_plan(current_loc, goal_loc)

        print(f"Plan created: {self.plan}")
        time.sleep(2)

        # 3. ACT
        # Execute the plan blindly (assuming the world doesn't change)
        for action in self.plan:
            self.env.update_agent_pos(action)
            self.env.display()

        print("Goal Reached!")


In [ ]:
world = Environment(5, (0, 0), (4, 4))
agent = GoalBasedAgent(world)

world.display()
agent.run()

--- SIMPLE GOAL AGENT ---
Goal: [4, 4] | Current: [0, 0]
----------------
| A             |
|               |
|               |
|               |
|             G |
----------------
Agent is formulating a plan based on coordinates...
Plan created: ['Down', 'Down', 'Down', 'Down', 'Right', 'Right', 'Right', 'Right']
--- SIMPLE GOAL AGENT ---
Goal: [4, 4] | Current: [1, 0]
----------------
|               |
| A             |
|               |
|               |
|             G |
----------------
--- SIMPLE GOAL AGENT ---
Goal: [4, 4] | Current: [2, 0]
----------------
|               |
| .             |
| A             |
|               |
|             G |
----------------
--- SIMPLE GOAL AGENT ---
Goal: [4, 4] | Current: [3, 0]
----------------
|               |
| .             |
| .             |
| A             |
|             G |
----------------
--- SIMPLE GOAL AGENT ---
Goal: [4, 4] | Current: [4, 0]
----------------
|               |
| .             |
| .             |
| .          

Here are the key concepts demonstrated in this specific implementation of a Goal-Based Agent:

1. Planning Before Acting (The Core Difference): Unlike a reflex agent that acts on impulse (e.g., "I see a wall, I turn left"), this agent uses formulate_plan to generate a complete sequence of actions (['Down', 'Down', ..., 'Right', 'Right']) before it ever moves its motors. It "thinks" through the entire problem first.

2. Explicit Goal Formulation: The agent calls formulate_goal() to define exactly what success looks like ((4,4)). All future actions are derived specifically to satisfy this target state.

3. Open-Loop Execution: In the run() method, once the plan is made, the agent executes it blindly:
```python
for action in self.plan:
    self.env.update_agent_pos(action)
```
This is known as "Open-Loop" control. The agent assumes the world won't change while it is moving. If an obstacle were suddenly dropped in its path after it started moving, this specific agent would crash because it isn't checking its sensors during the execution phase.

4. World Representation (Coordinates): The agent understands the world through coordinates. Its "intelligence" here is simple arithmetic: calculating the vertical difference (goal[0] - start[0]) and horizontal difference (goal[1] - start[1]) to determine the necessary steps. This is a very basic form of a heuristic